# Reddit r/WritingPrompts data downloader

In [4]:
# Step 2: Import libraries
import praw
import pandas as pd
import time
import os

In [5]:
# Step 3: Set up Reddit API credentials
reddit = praw.Reddit(
    client_id='-eqmyJxGZcoKmesk8lZzdQ',
    client_secret='tDsl-SGg5_ZSNSmoHtlvTwGwxkjSrw',
    user_agent='-eqmyJxGZcoKmesk8lZzdQ'
)

In [6]:
# Step 4: Choose a subreddit and fetch posts and comments
subreddit_name = "WritingPrompts"
post_limit = 1000  # Number of posts to scrape

subreddit = reddit.subreddit(subreddit_name)

posts_data = []
comments_data = []

try:
    for index, post in enumerate(subreddit.hot(limit=post_limit)):
        if post.title[:4] != '[WP]':
            continue

        posts_data.append({
            'title': post.title,
            'id': post.id,
            'num_comments': post.num_comments
        })

        try:
            post.comments.replace_more(limit=0)
            for comment in post.comments.list():
                if comment.score < 5 or len(comment.body) < 256:
                    continue
                comments_data.append({
                    'post_id': post.id,
                    'comment_id': comment.id,
                    'comment_body': comment.body,
                    'comment_score': comment.score
                })

        except Exception as e:
            print(f"Error processing comments for post {post.id}: {e}")


except Exception as e:
    print(f"An unexpected error occurred: {e}")
    time.sleep(5)

# Create DataFrames
posts_df = pd.DataFrame(posts_data)
comments_df = pd.DataFrame(comments_data)

In [7]:
print(len(posts_df))
posts_df.head()

841


,title,id,num_comments
0,[WP] You’re the villain in a prophecy—but the ...,1lqha2t,10
1,[WP] a powerful wizard who has taken in severa...,1lqq5lg,2
2,[WP] You are a supervillain. You've discovered...,1lq4q3k,16
3,"[WP] ""BE NOT AFRAID-"" ""Hey, uh, respectfully, ...",1lqm0z0,3
4,[WP] There's an app on your phone that you don...,1lqql6d,2


In [8]:
print(len(comments_df))
comments_df.head()

815


,post_id,comment_id,comment_body,comment_score
0,1lqha2t,n12vob3,"Okay, I rose from a peasant girl, to a Duchess...",113
1,1lq4q3k,n1003np,"I had worked my way up the chain, putting in m...",242
2,1lq4q3k,n11drei,KRRRRT. KRRRRT. \n\nBlake’s eyes were wide ope...,75
3,1lq4q3k,n12dac6,I found Miss Marvelous home address. I zoomed ...,6
4,1lqm0z0,n13q9a9,The thing with a random face in the middle of ...,12


In [9]:
# Step 6: Save the data
posts_df.to_csv("../data/raw/reddit_posts_03072025.csv", index=False)
comments_df.to_csv("../data/raw/reddit_comments_03072025.csv", index=False)

print("Data saved to reddit_posts.csv and reddit_comments.csv")

Data saved to reddit_posts.csv and reddit_comments.csv


In [10]:
# Have to finish pretraining the model today and trying it out. for sure.
# Lets try and split into training and validation sets
training_df = pd.DataFrame(columns=['Post_ID', 'Comment_ID', 'Prompt', 'Story', 'Story_Score'])
validation_df = pd.DataFrame(pd.DataFrame(columns=['Post_ID', 'Prompt']))

for index, post in posts_df.iterrows():
    if post.id in comments_df.post_id.values:
        comments = comments_df[comments_df.post_id == post.id]
        for index, comment in comments.iterrows():
            training_value = {
                'Post_ID': post.id,
                'Comment_ID': comment.comment_id,
                'Prompt': post.title[4:],  # Remove '[WP]' from the title
                'Story': comment.comment_body,
                'Story_Score': comment.comment_score
            }
            training_df = pd.concat([training_df, pd.DataFrame([training_value])], ignore_index=True)
    else:
        validation_value = {
            'Post_ID': post.id,
            'Prompt': post.title
        }
        validation_df = pd.concat([validation_df, pd.DataFrame([validation_value])], ignore_index=True) 

In [11]:
print(len(training_df))
training_df.head()

815


,Post_ID,Comment_ID,Prompt,Story,Story_Score
0,1lqha2t,n12vob3,You’re the villain in a prophecy—but the hero...,"Okay, I rose from a peasant girl, to a Duchess...",113
1,1lq4q3k,n1003np,You are a supervillain. You've discovered whe...,"I had worked my way up the chain, putting in m...",242
2,1lq4q3k,n11drei,You are a supervillain. You've discovered whe...,KRRRRT. KRRRRT. \n\nBlake’s eyes were wide ope...,75
3,1lq4q3k,n12dac6,You are a supervillain. You've discovered whe...,I found Miss Marvelous home address. I zoomed ...,6
4,1lqm0z0,n13q9a9,"""BE NOT AFRAID-"" ""Hey, uh, respectfully, you ...",The thing with a random face in the middle of ...,12


In [12]:
print(len(validation_df))
validation_df.head()

549


,Post_ID,Prompt
0,1lqq5lg,[WP] a powerful wizard who has taken in severa...
1,1lqql6d,[WP] There's an app on your phone that you don...
2,1lqj12o,[WP] At a monastery where many take vows of si...
3,1lqo361,[WP] A fictional character comes to life with ...
4,1lqqxbj,[WP] Life is all about the simple things. A go...
